In [1]:
#!pip install sklearn_crfsuite
#!pip install eli5
# !pip install WordCloud
!pip install scikit-learn==0.23

# Import necessary libraries

In [2]:
import json
import pandas as pd
import numpy as np
import sklearn
from pandas.core.groupby.base import OutputKey
from sklearn.feature_extraction import DictVectorizer
import sklearn_crfsuite
from sklearn_crfsuite import scorers, metrics
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
import nltk
from tqdm import tqdm
import json
import eli5
import spacy
from spacy.training import offsets_to_biluo_tags
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import csv
import re
from spacy.training import Example
from spacy.tokens import DocBin
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator

In [3]:
#load dataset
df_train = pd.read_csv("Final_dataset.csv")

In [4]:
test_data = [json.loads(line) for line in open('validation.json','r')]
test_data = pd.json_normalize(test_data)
test_data = test_data['Name']
test_data = test_data[:1000]

In [5]:
df_train.shape

(2393, 3)

In [6]:
#Only product titles are needed
df_train = df_train['Title']

In [7]:
df_train.head()

0                          Sterling Silver Angel Charm
1        HP Pavilion 23xi 58.40 cm (23ï¿½) IPS Monitor
2    East Carolina Pirates Ladies Personalized Bask...
3    Tekonsha 90195 P3 Electric Brake Control, 1-4 ...
4                    RN-XV WiFly Module - Wire Antenna
Name: Title, dtype: object

In [8]:
#nlp = spacy.load("en_core_web_lg")

In [9]:
#define custom functions to load, save and generate product and brand names list
def load_data(file):
    with open(file, "r", encoding = "utf-8")as f:
        data = json.load(f)
    return data

def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
        
def generate_brand_products(file):
    chars = load_data(file)
    #print(len(hp_chars))
    #print(chars)
    new_characters = []
    for item in chars:
        new_characters.append(item.lower())
    return new_characters

In [10]:
brands = generate_brand_products("brand_names.json")

In [11]:
products = generate_brand_products("products.json")

In [12]:
print(brands)

['ruby reds', 'victorian', 'michigan state spartans', 'under armour', 'drake bulldogs', 'modernus', 'homedics', 'adidas', 'cleveland browns', 'harpa', 'guanabana', "nature's promise", 'chrysler', 'tech air', 'east carolina pirates', 'oase', 'troy', 'el gran', 'iowa state cyclones', 'reloj', 'northern illinois', 'bella', 'sellier & bellot', 'cammo', 'boston bruins', 'nike', 'eden', 'chevrolet', 'alterna', 'plyboo', 'anatolius', 'scientific anglers', 'alabama', 'purdue', 'maxtens', 'brand', 'sterling', 'hp', 'east carolina pirates', 'tekonsha', 'wifly', 'genairex', 'philadelphia eagles ', 'kansas city chiefs', 'goldwax', 'nittaku', 'oakland athletics', 'thule', 'creighton bluejays', 'seattle seahawks', 'vintage', 'arizona wildcats', 'miami hurricanes', 'kentucky wildcats', 'bo jackson', 'mariposa', 'alabama', 'multi-stripe', 'nike', 'dayton flyers', 'louisiana', 'michigan state', 'helena', 'grizzly straw', 'sam houston', 'seminoles', 'aggies', 'duke', 'colorblock', 'archie griffin', ' ho

In [13]:
print(products)

['locket', 'monitor', 'tshirt', 'display case', 'barwell fleet', 'plaque', 'hoodie', 'garden flag', 'dinning chair', 'hoodie', 'tshirt', 'football', 'sunglasses', 'ornament', 'money clip', 'flip flops', 'dress', 'hat', 'barrette', 'hats', 'photograph', 'tank top', 'soundtrack', 'socks', 'jersey', 'guillotine', 'pills', 'tote', 'sweatshirt', 'bags', 'sticker', 'barware', 'bulb kit', 'mold', 'polo', 'cubby', 'tstat', 'shirt', 'decal', 'cleats', 'gift set', 'scrub', 'dog dry', 'expedition pack', 'frame', 'pizelle maker', 'shuffler', 'eye cream', 'pineapple', 'brushguard', 'bowl', 'teakettle', 'lighter', 'map 2012', 'training', 'lid', 'jacket', 'hard disk', 'dog', 'flooring', 'tag', 'cape', 'diamonds', 'cleaner', 'shoes', 'powder', 'extension', 'tie', 'backpack', 'touring', 'riflescope', 'accessories', 'civic lx', 'netlimited', 'sign', 'skirt', 'repisa', 'marker', 'pad', 'slipper', 'card', 'sugar', 'overalls', 'tee', 'oak', 'pump', 'thermal halfzip', 'set', 'playlist', 'kit', 'necklace', '

# Spacy customization

In [14]:
#function to create input data for entity ruler
def create_training_data(file, entity_type):
    data = generate_brand_products(file)
    patterns = []
    for item in data:
        pattern = {
                    "label": entity_type,
                    "pattern": item
                  }
        patterns.append(pattern)
#     print(patterns)
    return patterns

In [15]:
patterns1 = create_training_data("brand_names.json", "BRAND")

In [16]:
patterns2 = create_training_data("products.json", "PRODUCT")

In [17]:
patterns1.extend(patterns2)

In [18]:
print(patterns1)

[{'label': 'BRAND', 'pattern': 'ruby reds'}, {'label': 'BRAND', 'pattern': 'victorian'}, {'label': 'BRAND', 'pattern': 'michigan state spartans'}, {'label': 'BRAND', 'pattern': 'under armour'}, {'label': 'BRAND', 'pattern': 'drake bulldogs'}, {'label': 'BRAND', 'pattern': 'modernus'}, {'label': 'BRAND', 'pattern': 'homedics'}, {'label': 'BRAND', 'pattern': 'adidas'}, {'label': 'BRAND', 'pattern': 'cleveland browns'}, {'label': 'BRAND', 'pattern': 'harpa'}, {'label': 'BRAND', 'pattern': 'guanabana'}, {'label': 'BRAND', 'pattern': "nature's promise"}, {'label': 'BRAND', 'pattern': 'chrysler'}, {'label': 'BRAND', 'pattern': 'tech air'}, {'label': 'BRAND', 'pattern': 'east carolina pirates'}, {'label': 'BRAND', 'pattern': 'oase'}, {'label': 'BRAND', 'pattern': 'troy'}, {'label': 'BRAND', 'pattern': 'el gran'}, {'label': 'BRAND', 'pattern': 'iowa state cyclones'}, {'label': 'BRAND', 'pattern': 'reloj'}, {'label': 'BRAND', 'pattern': 'northern illinois'}, {'label': 'BRAND', 'pattern': 'bella

In [19]:
#blank spacy English model with entity ruler in pipeline
def generate_spacy_rules(patterns):
    nlp = English()
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
#     print(nlp.pipe_names)
    nlp.to_disk("prod_ner")

In [20]:
generate_spacy_rules(patterns1)

In [21]:
nlp = spacy.load("prod_ner")

In [22]:
# nlp = spacy.load("prod_ner_crf")
# def test_model(model, text):
#     doc = nlp(text)
# #     print(text)
#     results = []
#     for ent in doc.ents:
#         results.append(ent.text)
# #     for token in doc:
# #         print(token.pos_)
# #     print(results)

In [23]:
#Function to format input data to required entity ruler format
def train_data_for_spacy(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        #print(ent.label_)
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) > 0:
        results = [text, {"entities": entities}]
        return (results)

In [24]:
#colors used in product titles listed
colors_list = load_data("colors.json")
colors = []
for item in colors_list:
    colors.append(item.lower())


In [25]:
a ='(23ã¯â¿â½)'
a.encode('ascii', 'ignore').decode()

'(23)'

# Spacy training

In [26]:
def get_spacy_train_data(input_data):
    nlp = spacy.load("prod_ner")
    DATA = []
    with open(input_data, "r", encoding='utf-8') as f:
        text = pd.read_csv(f)
#         print(text)
        df1 = list(text["Title"])
        data = []
        for item in df1:
            #print(item)
            data.append(item.lower())
        #print(data)
        for segment in data:
    #         print(segment)
            segment = segment.strip()
            segment = segment.replace("\n", " ")
            segment = segment.encode('ascii', 'ignore').decode()
            for i, word in enumerate(segment.split()):
    #             print(word)
                if word in STOP_WORDS:
    #                 print(word)
                    segment = segment.replace(word, "")
    #                 print(segment)
                if word in colors:
                    segment = segment.replace(word, "")

            segment = re.sub(' +', ' ', segment)
    #         print(segment)
            punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''        
            for ele in segment:
                #print(ele)
                if ele in punc:
                    segment = segment.replace(ele, "")
                    segment = segment.strip()

    #         print(len(segment.split()))
            len1 = len(segment.split())
            for i, word in enumerate(segment.split()):
                if i < len1-1:
                    if (word + ' ' + segment.split()[i+1]) in colors:
                        segment = segment.replace(word + " " + segment.split()[i+1] + "", "") 
                        segment = segment.strip("  ")

            results = train_data_for_spacy(nlp, segment.strip())
            if results != None:
                    DATA.append(results)
    return DATA

In [27]:
TRAIN_DATA = get_spacy_train_data("Final_dataset.csv")

In [28]:
#Save the training data
save_data("training_data.json", TRAIN_DATA)

In [29]:
TRAIN_DATA[:5]

[['sterling angel charm',
  {'entities': [(0, 8, 'BRAND'), (15, 20, 'PRODUCT')]}],
 ['hp pavilion 23xi 5840 cm 23 ips monitor',
  {'entities': [(0, 2, 'BRAND'), (32, 39, 'PRODUCT')]}],
 ['east carolina pirates ladies personalized basketball long sleeve classic fit tshirt',
  {'entities': [(0, 21, 'BRAND'), (42, 52, 'PRODUCT'), (77, 83, 'PRODUCT')]}],
 ['tekonsha 90195 p3 electric brake control 14 trailer axles',
  {'entities': [(0, 8, 'BRAND'), (52, 57, 'PRODUCT')]}],
 ['rnxv wifly module  wire antenna', {'entities': [(5, 10, 'BRAND')]}]]

In [30]:
#define a function to convert entities for BIO tagging
def convert_to_bio_tags(data):
    nlp = spacy.load('en_core_web_lg')
    docs = []
    train_bio = []
    k = 0
    for text, annot in data:
        doc = nlp(text)
        tags = offsets_to_biluo_tags(doc, annot['entities'])
        bio_tags = [tag.replace('L-BRAND','I-BRAND').replace('U-BRAND','B-BRAND').replace('L-PRODUCT','I-PRODUCT').replace('U-PRODUCT','B-PRODUCT') for tag in tags]
        tokens = [tok.text for tok in doc]
        postags = [token.pos_ for token in doc]
        token_tags = list(zip(tokens,postags, bio_tags))
        train_bio_sample = [{'sentence_id':k,'words':el[0],'POS':el[1], 'labels':el[2]} for el in token_tags]
        #print(train_bio_sample)
        train_bio += train_bio_sample
        k += 1 
    return pd.DataFrame(train_bio,columns=['sentence_id','words','POS','labels'])

In [31]:
#convert to BIO tags
all_bio_df = convert_to_bio_tags(TRAIN_DATA)

In [32]:
all_bio_df.isnull().sum()

sentence_id    0
words          0
POS            0
labels         0
dtype: int64

In [33]:
all_bio_df.groupby('labels').size().reset_index(name='counts')

,labels,counts
0,B-BRAND,3096
1,B-PRODUCT,3094
2,I-BRAND,2071
3,I-PRODUCT,199
4,O,10387


In [34]:
#get the uniwue labels
y = all_bio_df.labels.values

classes = np.unique(y)
classes = classes.tolist()

new_classes = classes.copy()
new_classes.pop()
new_classes

['B-BRAND', 'B-PRODUCT', 'I-BRAND', 'I-PRODUCT']

# CRF model

In [35]:
#function to gett input format
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['words'].values.tolist(),
                                                           s['POS'].values.tolist(),
                                                           s['labels'].values.tolist())]
        self.grouped = self.data.groupby('sentence_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [36]:
#genarate test data
test_data = pd.DataFrame(test_data)
test_data.rename(columns={'Name':'Title'}, inplace=True)

test_data.to_csv("test_data.csv")

TEST_DATA = get_spacy_train_data('test_data.csv')

In [37]:
TEST_DATA[1]

['mens michigan state spartans big logo hoodie',
 {'entities': [(5, 28, 'BRAND'), (33, 37, 'PRODUCT'), (38, 44, 'PRODUCT')]}]

In [38]:
test_df = convert_to_bio_tags(TEST_DATA)
test_df.head()

,sentence_id,words,POS,labels
0,0,victorian,ADJ,B-BRAND
1,0,masonic,PROPN,O
2,0,locket,NOUN,B-PRODUCT
3,0,1890,NUM,O
4,1,mens,PROPN,O


In [39]:
getter = SentenceGetter(test_df)
test_sent = getter.sentences

In [40]:
getter = SentenceGetter(all_bio_df)
train_sent = getter.sentences

In [41]:
#Feature extraction and generation for CRF model
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'word': word,
        'bias': 1.0, 
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
#             'previous word': word1,
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isdigit()': word.isdigit(),
            '-1:postag': postag1,
        })
    else:
        features['BeginingOfSentence'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
#             'next word': word1,
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isdigit()': word.isdigit(),
            '+1:postag': postag1,
        })
    else:
        features['EndOfSentence'] = True
    return features

In [42]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, pos, label in sent]

def sent2tokens(sent):
    return [token for token, pos, label in sent]

### CRF model with word and pos as features

In [43]:
X = [sent2features(s) for s in train_sent]
y = [sent2labels(s) for s in train_sent]


In [44]:
X[0]

[{'word': 'sterling',
  'bias': 1.0,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'ADJ',
  'BeginingOfSentence': True,
  '+1:word.istitle()': False,
  '+1:word.isdigit()': False,
  '+1:postag': 'NOUN'},
 {'word': 'angel',
  'bias': 1.0,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NOUN',
  '-1:word.istitle()': False,
  '-1:word.isdigit()': False,
  '-1:postag': 'ADJ',
  '+1:word.istitle()': False,
  '+1:word.isdigit()': False,
  '+1:postag': 'NOUN'},
 {'word': 'charm',
  'bias': 1.0,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NOUN',
  '-1:word.istitle()': False,
  '-1:word.isdigit()': False,
  '-1:postag': 'NOUN',
  'EndOfSentence': True}]

In [45]:
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]

In [46]:
#define parameters for CRF model
parameters = {'c1':[0.1, 0.2, 0.5, 1], 'c2':[0.1, 0.2, 0.5, 1], 'max_iterations': [100, 150, 200]}

In [47]:
#CRF modelling
from sklearn_crfsuite.metrics import flat_accuracy_score, flat_f1_score
for c1_val in parameters['c1']:
    for c2_val in parameters['c2']:
        for iters in parameters['max_iterations']:
            print(f"c1 value : {c1_val} , c2 value : {c2_val} , iter values : {iters}")
            crf = sklearn_crfsuite.CRF(
                algorithm='lbfgs',
                all_possible_transitions=True,
                c1 = c1_val,
                c2 = c2_val,
                max_iterations = iters)
            crf.fit(X, y)
            y_pred = crf.predict(X_test)
            print(flat_accuracy_score(y_test, y_pred))
            print(flat_f1_score(y_test, y_pred, average = 'weighted'))

c1 value : 0.1 , c2 value : 0.1 , iter values : 100
0.9309221803222489
0.9329246705993782
c1 value : 0.1 , c2 value : 0.1 , iter values : 150
0.9273225917038053
0.9292793786477085
c1 value : 0.1 , c2 value : 0.1 , iter values : 200
0.9274940006856359
0.9294399748983977
c1 value : 0.1 , c2 value : 0.2 , iter values : 100
0.914809736030168
0.9166991718664941
c1 value : 0.1 , c2 value : 0.2 , iter values : 150
0.9149811450119987
0.9168767238394105
c1 value : 0.1 , c2 value : 0.2 , iter values : 200
0.914295509084676
0.916191105994524
c1 value : 0.1 , c2 value : 0.5 , iter values : 100
0.8863558450462804
0.8876507863338726
c1 value : 0.1 , c2 value : 0.5 , iter values : 150
0.8861844360644497
0.8875170457241417
c1 value : 0.1 , c2 value : 0.5 , iter values : 200
0.8858416181007884
0.8871941321165812
c1 value : 0.1 , c2 value : 1 , iter values : 100
0.8676722660267397
0.8680284316446857
c1 value : 0.1 , c2 value : 1 , iter values : 150
0.8668152211175866
0.8671041575202102
c1 value : 0.1 , 

In [48]:
from sklearn_crfsuite.metrics import flat_accuracy_score, flat_f1_score
from sklearn_crfsuite import CRF
wcrf_pos = CRF(algorithm='lbfgs',
                           all_possible_transitions=True,
                           c1 = 0.1,
                           c2 = 0.1,
                           max_iterations = 100)
wcrf_pos.fit(X, y)
y_pred = wcrf_pos.predict(X_test)

print(metrics.flat_classification_report(y_test, y_pred))
print('Accuracy: ',flat_accuracy_score(y_test, y_pred))
print('F1-Score: ',flat_f1_score(y_test, y_pred, average = 'weighted'))

              precision    recall  f1-score   support

     B-BRAND       0.80      0.93      0.86       676
   B-PRODUCT       0.98      0.92      0.95       991
     I-BRAND       0.86      0.97      0.91       494
   I-PRODUCT       0.50      0.93      0.65        42
           O       0.97      0.93      0.95      3631

    accuracy                           0.93      5834
   macro avg       0.82      0.94      0.86      5834
weighted avg       0.94      0.93      0.93      5834

Accuracy:  0.9309221803222489
F1-Score:  0.9329246705993782


C:\Users\nitee\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [49]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(wcrf_pos, X, y, cv=5)
print("Cross-validation Accuracy: ", scores.mean())

C:\Users\nitee\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


Cross-validation Accuracy:  0.9257970775257389


In [52]:
len(y_pred)

795

### Wrong Tag error analysis

In [56]:
ypred = [y for tok in item for item in y_pred]

In [57]:
len(ypred)

3180

In [58]:
wrong_tag = 0
for gold, pred in zip(y_test, y_pred):
    for i in range(len(gold)):
#         print(gold[2])
#         print(pred[2])
        if (gold[i]==pred[i])== False:
            wrong_tag += 1

print(wrong_tag)

403


In [59]:
wrong_tag_perc = wrong_tag/len(ypred)
wrong_tag_perc

0.12672955974842767

### No Annotation error analysis

In [62]:
total = 0
for pred in y_pred:
#     print(pred)
    for i in range(len(pred)):
#         print(pred[i])
#         if (str(pred[i]) in tags):
#             total =+ 1
        if (pred[i]=='B-BRAND' or pred[i]=='I-BRAND' or pred[i]=='B-PRODUCT' or pred[i]=='I-PRODUCT'):
            total +=1

print(total)

2357


In [63]:
no_annot = 0
for gold, pred in zip(y_test, y_pred):
    for i in range(len(gold)):
#         print(gold[2])
#         print(pred[2])
        if gold[i]=='B-BRAND':
            if pred[i]=="O":
                no_annot += 1
        if gold[i]=='B-PRODUCT':
            if pred[i]=="O":
                no_annot += 1
        if gold[i]=='I-BRAND':
            if pred[i]=="O":
                no_annot += 1
        if gold[i]=='I-PRODUCT':
            if pred[i]=="O":
                no_annot += 1
                
print(no_annot)

111


In [64]:
no_annotation = no_annot/total
no_annotation

0.047093763258379294

### CRF with word as feature but no pos

In [49]:
#Feature extraction and generation for CRF model without pos tags
def word2features(sent, i):
    word = sent[i][0]
#     postag = sent[i][1]
    features = {
        'word': word,
        'bias': 1.0, 
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
#         'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
#         postag1 = sent[i-1][1]
        features.update({
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isdigit()': word.isdigit(),
#             '-1:postag': postag1,
        })
    else:
        features['BeginingOfSentence'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
#         postag1 = sent[i+1][1]
        features.update({
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isdigit()': word.isdigit(),
#             '+1:postag': postag1,
        })
    else:
        features['EndOfSentence'] = True
    return features

#train test split
X = [sent2features(s) for s in train_sent]
y = [sent2labels(s) for s in train_sent]

In [50]:
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]

In [65]:
#CRF modelling
for c1_val in parameters['c1']:
    for c2_val in parameters['c2']:
        for iters in parameters['max_iterations']:
            print(f"c1 value : {c1_val} , c2 value : {c2_val} , iter values : {iters}")
            crf = sklearn_crfsuite.CRF(
                algorithm='lbfgs',
                all_possible_transitions=True,
                c1 = c1_val,
                c2 = c2_val,
                max_iterations = iters)
            crf.fit(X, y)
            y_pred = crf.predict(X_test)
            print(flat_accuracy_score(y_test, y_pred))
            print(flat_f1_score(y_test, y_pred, average = 'weighted'))

c1 value : 0.1 , c2 value : 0.1 , iter values : 100
0.9266369557764826
0.9295329932955896
c1 value : 0.1 , c2 value : 0.1 , iter values : 150
0.926979773740144
0.9299038948122776
c1 value : 0.1 , c2 value : 0.1 , iter values : 200
0.926979773740144
0.9299038948122776
c1 value : 0.1 , c2 value : 0.2 , iter values : 100
0.9149811450119987
0.9176860054007099
c1 value : 0.1 , c2 value : 0.2 , iter values : 150
0.9149811450119987
0.9176860054007099
c1 value : 0.1 , c2 value : 0.2 , iter values : 200
0.9149811450119987
0.9176860054007099
c1 value : 0.1 , c2 value : 0.5 , iter values : 100
0.8969832019197806
0.8987820860915295
c1 value : 0.1 , c2 value : 0.5 , iter values : 150
0.8969832019197806
0.8987820860915295
c1 value : 0.1 , c2 value : 0.5 , iter values : 200
0.8969832019197806
0.8987820860915295
c1 value : 0.1 , c2 value : 1 , iter values : 100
0.8486458690435379
0.8510918996775109
c1 value : 0.1 , c2 value : 1 , iter values : 150
0.8486458690435379
0.8510978063559991
c1 value : 0.1 ,

In [51]:
#c1 value : 0.2 , c2 value : 0.1 , iter values : 100
from sklearn_crfsuite.metrics import flat_accuracy_score, flat_f1_score
wcrf_wo_pos = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           all_possible_transitions=True,
                           c1 = 0.1,
                           c2 = 0.1,
                           max_iterations = 150)
wcrf_wo_pos.fit(X, y)
y_pred = wcrf_wo_pos.predict(X_test)

print(metrics.flat_classification_report(y_test, y_pred))
print('Accuracy: ',flat_accuracy_score(y_test, y_pred))
print('F1-Score: ',flat_f1_score(y_test, y_pred, average = 'weighted'))

              precision    recall  f1-score   support

     B-BRAND       0.76      0.94      0.84       676
   B-PRODUCT       0.99      0.92      0.95       991
     I-BRAND       0.91      0.96      0.93       494
   I-PRODUCT       0.44      0.90      0.59        42
           O       0.97      0.92      0.94      3631

    accuracy                           0.93      5834
   macro avg       0.81      0.93      0.85      5834
weighted avg       0.94      0.93      0.93      5834

Accuracy:  0.926979773740144
F1-Score:  0.9299038948122776


In [73]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(wcrf_wo_pos, X, y, cv=5)
print("Cross-validation Accuracy: ", scores.mean())

Cross-validation Accuracy:  0.9287954863018412


### CRF without word as feature but including pos as feature

In [52]:
#Feature extraction and generation for CRF model
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
#         'word': word,
        'bias': 1.0, 
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isdigit()': word.isdigit(),
            '-1:postag': postag1,
        })
    else:
        features['BeginingOfSentence'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isdigit()': word.isdigit(),
            '+1:postag': postag1,
        })
    else:
        features['EndOfSentence'] = True
    return features

In [53]:
X = [sent2features(s) for s in train_sent]
y = [sent2labels(s) for s in train_sent]

In [54]:
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]

In [59]:
#CRF modelling
for c1_val in parameters['c1']:
    for c2_val in parameters['c2']:
        for iters in parameters['max_iterations']:
            print(f"c1 value : {c1_val} , c2 value : {c2_val} , iter values : {iters}")
            crf = sklearn_crfsuite.CRF(
                algorithm='lbfgs',
                all_possible_transitions=True,
                c1 = c1_val,
                c2 = c2_val,
                max_iterations = iters)
            crf.fit(X, y)
            y_pred = crf.predict(X)
            print(metrics.flat_classification_report(y, y_pred))
#             print(flat_accuracy_score(y_test, y_pred))
#             print(flat_f1_score(y_test, y_pred, average = 'weighted'))

c1 value : 0.1 , c2 value : 0.1 , iter values : 100
              precision    recall  f1-score   support

     B-BRAND       0.63      0.45      0.52      3096
   B-PRODUCT       0.75      0.59      0.66      3094
     I-BRAND       0.41      0.45      0.43      2071
   I-PRODUCT       0.00      0.00      0.00       199
           O       0.73      0.84      0.78     10387

    accuracy                           0.68     18847
   macro avg       0.50      0.46      0.48     18847
weighted avg       0.67      0.68      0.67     18847

c1 value : 0.1 , c2 value : 0.1 , iter values : 150
              precision    recall  f1-score   support

     B-BRAND       0.63      0.45      0.52      3096
   B-PRODUCT       0.75      0.59      0.66      3094
     I-BRAND       0.41      0.45      0.43      2071
   I-PRODUCT       0.00      0.00      0.00       199
           O       0.73      0.84      0.78     10387

    accuracy                           0.68     18847
   macro avg       0.50    

              precision    recall  f1-score   support

     B-BRAND       0.63      0.45      0.52      3096
   B-PRODUCT       0.75      0.59      0.66      3094
     I-BRAND       0.41      0.45      0.43      2071
   I-PRODUCT       0.00      0.00      0.00       199
           O       0.73      0.84      0.78     10387

    accuracy                           0.68     18847
   macro avg       0.50      0.46      0.48     18847
weighted avg       0.67      0.68      0.67     18847

c1 value : 0.2 , c2 value : 0.2 , iter values : 200
              precision    recall  f1-score   support

     B-BRAND       0.63      0.45      0.52      3096
   B-PRODUCT       0.75      0.59      0.66      3094
     I-BRAND       0.41      0.45      0.43      2071
   I-PRODUCT       0.00      0.00      0.00       199
           O       0.73      0.84      0.78     10387

    accuracy                           0.68     18847
   macro avg       0.50      0.46      0.48     18847
weighted avg       0.67  

              precision    recall  f1-score   support

     B-BRAND       0.63      0.44      0.52      3096
   B-PRODUCT       0.75      0.59      0.66      3094
     I-BRAND       0.39      0.39      0.39      2071
   I-PRODUCT       0.00      0.00      0.00       199
           O       0.72      0.84      0.78     10387

    accuracy                           0.68     18847
   macro avg       0.50      0.45      0.47     18847
weighted avg       0.67      0.68      0.67     18847

c1 value : 0.5 , c2 value : 1 , iter values : 100
              precision    recall  f1-score   support

     B-BRAND       0.63      0.44      0.52      3096
   B-PRODUCT       0.75      0.59      0.66      3094
     I-BRAND       0.39      0.39      0.39      2071
   I-PRODUCT       0.00      0.00      0.00       199
           O       0.72      0.84      0.78     10387

    accuracy                           0.68     18847
   macro avg       0.50      0.45      0.47     18847
weighted avg       0.67    

In [60]:
#c1 value : 0.2 , c2 value : 0.1 , iter values : 100
from sklearn_crfsuite.metrics import flat_accuracy_score, flat_f1_score
crf_pos = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           all_possible_transitions=True,
                           c1 = 0.2,
                           c2 = 0.1,
                           max_iterations = 100)
crf_pos.fit(X, y)
y_pred = crf_pos.predict(X)

print(metrics.flat_classification_report(y, y_pred))
print('Accuracy: ',flat_accuracy_score(y, y_pred))
print('F1-Score: ',flat_f1_score(y, y_pred, average = 'weighted'))

              precision    recall  f1-score   support

     B-BRAND       0.63      0.45      0.52      3096
   B-PRODUCT       0.75      0.59      0.66      3094
     I-BRAND       0.41      0.45      0.43      2071
   I-PRODUCT       0.00      0.00      0.00       199
           O       0.73      0.84      0.78     10387

    accuracy                           0.68     18847
   macro avg       0.50      0.46      0.48     18847
weighted avg       0.67      0.68      0.67     18847

Accuracy:  0.6809571815142994
F1-Score:  0.6705404427448096


In [61]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(crf_pos, X, y, cv=5)
print("Cross-validation Accuracy: ", scores.mean())

C:\Users\nitee\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


Cross-validation Accuracy:  0.6777460573553377


### CRF without word and pos as feature

In [62]:
#Feature extraction and generation for CRF model without pos tags
def word2features(sent, i):
    word = sent[i][0]
#     postag = sent[i][1]
    features = {
#         'word': word,
        'bias': 1.0, 
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
#         'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
#         postag1 = sent[i-1][1]
        features.update({
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isdigit()': word.isdigit(),
#             '-1:postag': postag1,
        })
    else:
        features['BeginingOfSentence'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
#         postag1 = sent[i+1][1]
        features.update({
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isdigit()': word.isdigit(),
#             '+1:postag': postag1,
        })
    else:
        features['EndOfSentence'] = True
    return features

#train test split
X = [sent2features(s) for s in train_sent]
y = [sent2labels(s) for s in train_sent]


In [63]:
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]

In [65]:
#c1 value : 0.2 , c2 value : 0.1 , iter values : 100
from sklearn_crfsuite.metrics import flat_accuracy_score, flat_f1_score
crf_wo_pos = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           all_possible_transitions=True,
                           c1 = 0.2,
                           c2 = 0.1,
                           max_iterations = 100)
crf_wo_pos.fit(X, y)
y_pred = crf_wo_pos.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

print('Accuracy: ',flat_accuracy_score(y_test, y_pred))
print('F1-Score: ',flat_f1_score(y_test, y_pred, average = 'weighted'))

              precision    recall  f1-score   support

     B-BRAND       0.49      0.51      0.50       676
   B-PRODUCT       0.60      0.47      0.53       991
     I-BRAND       0.31      0.44      0.36       494
   I-PRODUCT       0.00      0.00      0.00        42
           O       0.73      0.74      0.73      3631

    accuracy                           0.63      5834
   macro avg       0.43      0.43      0.42      5834
weighted avg       0.64      0.63      0.63      5834

Accuracy:  0.633870414809736
F1-Score:  0.6346538716095479


C:\Users\nitee\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [83]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(crf_wo_pos, X, y, cv=5)
print("Cross-validation Accuracy: ", scores.mean())

C:\Users\nitee\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


Cross-validation Accuracy:  0.6484518591129635
